In [ ]:
!git clone https://github.com/WongKinYiu/yolov7
%cd yolov7
!pip install -r requirements.txt

In [ ]:
import os
from PIL import Image
import torchvision.transforms as transforms
import cv2
import numpy as np
import torch

def preprocess_for_ddpm(image_path, img_size=256):
    transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    image = Image.open(image_path).convert("RGB")
    tensor = transform(image).unsqueeze(0)
    return tensor

def preprocess_for_yolo(image_path, img_size=640):
    image = cv2.imread(image_path)
    h, w = image.shape[:2]
    scale = img_size / max(h, w)
    new_w, new_h = int(w * scale), int(h * scale)
    resized = cv2.resize(image, (new_w, new_h))

    top = (img_size - new_h) // 2
    bottom = img_size - new_h - top
    left = (img_size - new_w) // 2
    right = img_size - new_w - left

    padded = cv2.copyMakeBorder(resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=(114, 114, 114))
    return padded

def preprocess_folder(folder_path, output_folder, ddpm=True, img_size=256):
    os.makedirs(output_folder, exist_ok=True)
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_path = os.path.join(folder_path, filename)
            if ddpm:
                tensor = preprocess_for_ddpm(input_path, img_size)
                torch.save(tensor, os.path.join(output_folder, filename.split('.')[0] + '.pt'))
            else:
                yolo_image = preprocess_for_yolo(input_path, img_size)
                cv2.imwrite(os.path.join(output_folder, filename), yolo_image)


In [ ]:
!python train.py --img-size 640 --batch-size 16 --epochs 50 --data /content/vessel_data/dataset.yaml --weights yolov7.pt --device 0